# Functions

Functions:
- enable you to break large programs into smaller, simpler pieces
- improve readability and make code more approachable
- allow for reuse and refactoring

## Item 14: Prefer Exceptions to returning `None`

It seems that `None` would make sense in some cases.

In [1]:
def divide(a, b):
    try:
        return a / b
    except ZeroDivisionError:
        return None

In [3]:
x, y = 2, 0
result = divide(x, y)
if result is None:
    print("Invalid input")

Invalid input


In [4]:
# But that can cause issues especially when confused
# with other falsey values

x, y = 0, 5
result = divide(x, y)
if not result:
    print("Invalid input")

Invalid input


In [6]:
# Split return value into a two-tuple

def divide(a, b):
    try:
        return True, a / b
    except ZeroDivisionError:
        return False, None

x, y = 2, 0
success, result = divide(x, y)
if not success:
    print('Invalid input')

Invalid input


In [7]:
x, y = 0, 5
_, result = divide(x, y)
if not result:
    print('Invalid input')

Invalid input


In [8]:
# Better way - never return None
# throw an exception

def divide(a, b):
    try:
        return a / b
    except ZeroDivisionError as e:
        raise ValueError('Invalid input') from e

In [10]:
x, y = 5, 2
try:
    result = divide(x, y)
except ValueError:
    print('Invalid input')
else:
    print('Result is %.1f' % result)

Result is 2.5


## Item 15: Know how closures interact with variable scope

If you want to sort a list of numbers, but prioritize a group of them to appear first, you can pass a helper function as a `key` argument to `sort`.

In [2]:
def sort_priority(values, group):
    def helper(x):
        if x in group:
            return (0, x)
        return (1, x)
    values.sort(key=helper)

In [3]:
numbers = [8, 3, 1, 2, 5, 4, 7, 6]
group = {2, 3, 5, 7}
sort_priority(numbers, group)
print(numbers)

[2, 3, 5, 7, 1, 4, 6, 8]


Three reasons why it works:
- Python supports closures - functions that refer to variables from the scope in which they were defined (helper is able to access `group` inside of `sort_priority`)
- functions are first-class objects - you can refer to them directly, assing them to varianles, pass them as arguments to other functions, compare them in expressions and `if` statements (this is how `sort` can accept a closure function as the `key`)
- Python has specific rules for comparing tuples - first items in index zero, then index one etc., this is why the return value from helper causes the sort order to have two distinct groups

It seems simple enough to check if a value has been seen at all.

In [14]:
def sort_priority_flag(values, group):
    found = False
    def helper(x):
        if x in group:
            found = True
            return (0, x)
        return (1, x)
    values.sort(key=helper)
    return found

found = sort_priority_flag(numbers, group)
print('Found: ', found)
print(numbers)

Found:  False
[2, 3, 5, 7, 1, 4, 6, 8]


Order of scope traversal to resolve a reference (looking for an existing variable to mutate/get the value of):
1. Current function's scope
2. Any enclosing scopes (other containing functions)
3. Scope of the module that contains the code (global scope)
4. Built-in scope (e.g. `len`, `str`)

If none of these places have a defined variable with the references name, then a `NameError` exception is raised.

For value assignment:
- if it's already in current scope, it will take on the new value
- if it doesn't exist in the current scope, then it's treated as a variable definition, in the function with the assignment

So the `found` variable in the `helper` closure is treated as a new variable definition, not assignment to the value that exists in the outer scope of `sort_priority_flag`.

### Why?
It prevents local variables in a function from polluting the containing module.

### How to get the value assignment to the outer scope?

There is the `nonlocal` syntax that indicates scope travesal should happen upon assignment for a specific variable name. It won't traverse up to the global scope.

It's compliment is the `global` scope that immediately assigns the variable in the module-level scope.

It might be useful for small pieces of code, but overall it becomes hard to track and reason about the bigger the code gets.

In [4]:
class Sorter(object):
    def __init__(self, group):
        self.group = group
        self.found = False
        
    def __call__(self, x):
        if x in self.group:
            self.found = True
            return (0, x)
        return (1, x)
    
sorter = Sorter(group)
numbers.sort(key=sorter)
assert sorter.found is True

In Python 2, `nonlocal` is not supported. A workaround is to mutate a list (dictionary/set/class instance) - instead of assigning a new variable, mutating the value in the outer scope.

```python
def sort_priority(numbers, group):
    found = [False]
    def helper(x):
        if x in group:
            found[0] = True
            return (0, x)
        return (1, x)
    numbers.sort(key=helper)
    return found[0]
```

## Item 16: Consider generators instead of returning lists

In [5]:
def index_words(text):
    result = []
    if text:
        result.append(0)
    for index, letter in enumerate(text):
        if letter == ' ':
            result.append(index + 1)
    return result

address = "Kitten puppy hamstah"
result = index_words(address)
print(result)

[0, 7, 13]


Opinions about the code:
- it's dense and noisy - "the method call's bulk deemphasizes the value being added" (I think this is a personal opinion more than it is a rule)
- there is one line for creating the result list and another for returning it, so while it contains ~130 characters, only ~75 are important (sheesh... I think it stands against the previous arguments in the book that longer is not worse)
- a better way to write it is using a generator, as it has less interactions with the result list
- the advantage of a generator is that not all the results have to be stored in the output list, which makes sense for big inputs

In [7]:
def index_words_iter(text):
    if text:
        yield 0
    for index, letter in enumerate(text):
        if letter == ' ':
            yield index + 1

# can convert to a list using the `list` built-in
result = list(index_words_iter(address))
print(result)

[0, 7, 13]


In [18]:
from itertools import islice

# generator with maximum input boundary
def index_file(handle):
    offset = 0
    for line in handle:
        if line:
            yield offset
        for letter in line:
            offset += 1
            if letter == ' ':
                yield offset

addresses = "kitten\npuppy\nhamstah"
print(index_file(addresses))
print(list(islice(index_file(addresses), 0, 5)))

<generator object index_file at 0x10f6744f8>
[0, 1, 2, 3, 4]
